## Pyvis, Gephi

In [3]:
import sys
sys.path.append("..")
from rcn_py import visual

In [26]:
visual.pyvis_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

In [4]:
visual.gephi_network_by_datafile('Stefan Verhoeven', "/Users/jennifer/RCSNA/data", "/Users/jennifer/RCSNA/visualization")

/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:206: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip('[')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:207: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  authors_orcid[i] = authors_orcid[i].strip(']')
/Users/jennifer/RCSNA/rcn-py/notebook/../rcn_py/data_storage.py:208: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  

## Neo4j to D3

In [2]:
from json import dumps
import logging
import os
from flask import (
    Flask,
    g,
    request,
    Response,
)
from neo4j import (
    GraphDatabase,
    basic_auth,
)

In [5]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "zhiningbai"

In [9]:
app = Flask(__name__, static_url_path="../static/")

# url = os.getenv("NEO4J_URI", uri)
# username = os.getenv("NEO4J_USER", "movies")
# password = os.getenv("NEO4J_PASSWORD", "movies")
# neo4j_version = os.getenv("NEO4J_VERSION", "4")
# database = os.getenv("NEO4J_DATABASE", "RCN DB")

port = os.getenv("PORT", 8080)

driver = GraphDatabase.driver(uri, auth=(username, password))

In [ ]:
# driver.session(database="neo4j")
def get_db():
    if not hasattr(g, "neo4j_db"):
        # if neo4j_version >= "4":
            g.neo4j_db = driver.session(database="neo4j")
        # else:
        #     g.neo4j_db = driver.session()
    return g.neo4j_db

@app.teardown_appcontext
def close_db(error):
    if hasattr(g, "neo4j_db"):
        g.neo4j_db.close()

@app.route("/")
def get_index():
    return app.send_static_file("index.html")

In [10]:
def serialize_person(person):
    return {
        "scopus_id": person["scopus_id"],
        "name": person["name"],
        "affiliation": person["affiliation"],
        "country": person["country"],
        "subject": person["subject"],
        "keywords": person["keywords"],
        "year": person["year"]
    }

def serialize_publication(pub):
    return {
        "doi": pub["doi"],
        "title": pub["title"],
        "subject": pub["subject"],
        "keywords": pub["keywords"],
        "year": pub["year"],
        "cited": pub["cited"]
    }

# def serialize_authorship(relationship):
#     return {
#         "author"
#     }

In [ ]:
# example: get pub-author relationships (2022, "Deep learning")
@app.route("/graph")
def get_example_graph():
    def work(tx, year, search_query, limit):
        return list(tx.run("""
            MATCH (n:Person)-[:IS_AUTHOR_OF]->(p:Publication (year: $year)) 
            WHERE $keyword in p.keywords
            RETURN n.name AS author,
                   p.title AS title
            LIMIT $limit
            """,
            year= year,
            keyword = search_query,
            limit = limit
        ))
    
    db = get_db()
    results = db.read_transaction(work, 
                                  request.args.get("year", 2022), 
                                  request.args.get("search_query", "Deep learning"), 
                                  request.args.get("limit", 300))
    nodes = []
    rels = []
    i = 0
    for record in results:
        nodes.append({"title": record["title"], "label": "publication"})
        target = i
        i += 1
        for name in record["author"]:
            author = {"title": name, "label": "author"}
            try:
                source = nodes.index(author)
            except ValueError:
                nodes.append(author)
                source = i
                i += 1
            rels.append({"source": source, "target": target})
    return Response(dumps({"nodes": nodes, "links": rels}),
                    mimetype="application/json")



    # try:
    #     q = request.args["q"]
    # except KeyError:
    #     return []
    # else:
    #     db = get_db()
    #     results = db.read_transaction(work, q)
    #     res_authors = Response(
    #         dumps([serialize_person(record["author"]) for record in results]),
    #         mimetype="application/json"
    #     )